## Cancellation rate. SQL

* Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03". Round Cancellation Rate to two decimal points.
* Return the result table in any order

The **cancellation rate** is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

In [1]:
import pandas as pd
import pandasql as ps

In [9]:
# Trips table:

# | id | client_id | driver_id | city_id | status              | request_at |
# +----+-----------+-----------+---------+---------------------+------------+
# | 1  | 1         | 10        | 1       | completed           | 2013-10-01 |
# | 2  | 2         | 11        | 1       | cancelled_by_driver | 2013-10-01 |
# | 3  | 3         | 12        | 6       | completed           | 2013-10-01 |
# | 4  | 4         | 13        | 6       | cancelled_by_client | 2013-10-01 |
# | 5  | 1         | 10        | 1       | completed           | 2013-10-02 |
# | 6  | 2         | 11        | 6       | completed           | 2013-10-02 |
# | 7  | 3         | 12        | 6       | completed           | 2013-10-02 |
# | 8  | 2         | 12        | 12      | completed           | 2013-10-03 |
# | 9  | 3         | 10        | 12      | completed           | 2013-10-03 |
# | 10 | 4         | 13        | 12      | cancelled_by_driver | 2013-10-03 |


In [7]:
# Users table:
# +----------+--------+--------+
# | users_id | banned | role   |
# +----------+--------+--------+
# | 1        | No     | client |
# | 2        | Yes    | client |
# | 3        | No     | client |
# | 4        | No     | client |
# | 10       | No     | driver |
# | 11       | No     | driver |
# | 12       | No     | driver |
# | 13       | No     | driver |
# +----------+--------+--------+

### Uploading tables as pandas DataFrames

In [21]:
trips = ''' 
 1  | 1         | 10        | 1       | completed           | 2013-10-01 
 2  | 2         | 11        | 1       | cancelled_by_driver | 2013-10-01 
 3  | 3         | 12        | 6       | completed           | 2013-10-01 
 4  | 4         | 13        | 6       | cancelled_by_client | 2013-10-01 
 5  | 1         | 10        | 1       | completed           | 2013-10-02 
 6  | 2         | 11        | 6       | completed           | 2013-10-02 
 7  | 3         | 12        | 6       | completed           | 2013-10-02 
 8  | 2         | 12        | 12      | completed           | 2013-10-03 
 9  | 3         | 10        | 12      | completed           | 2013-10-03 
 10 | 4         | 13        | 12      | cancelled_by_driver | 2013-10-03 
 '''

In [22]:
[row.replace(' ', '').split('|') for row in trips.split('\n')]

[['1', '1', '10', '1', 'completed', '2013-10-01'],
 ['2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'],
 ['3', '3', '12', '6', 'completed', '2013-10-01'],
 ['4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'],
 ['5', '1', '10', '1', 'completed', '2013-10-02'],
 ['6', '2', '11', '6', 'completed', '2013-10-02'],
 ['7', '3', '12', '6', 'completed', '2013-10-02'],
 ['8', '2', '12', '12', 'completed', '2013-10-03'],
 ['9', '3', '10', '12', 'completed', '2013-10-03'],
 ['10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03']]

In [23]:
trips = pd.DataFrame([row.replace(' ', '').split('|') for row in trips.split('\n')])

In [24]:
trips

,0,1,2,3,4,5
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [29]:
trips_columns = 'id | client_id | driver_id | city_id | status              | request_at'.split(' | ')
trips_columns = [name.strip() for name in trips_columns]

In [30]:
trips_columns

['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']

In [31]:
trips.columns = trips_columns

In [32]:
trips

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [37]:
users = ''' 
 1        | No     | client 
 2        | Yes    | client 
 3        | No     | client 
 4        | No     | client 
 10       | No     | driver 
 11       | No     | driver 
 12       | No     | driver 
 13       | No     | driver 
 '''

In [44]:
users = pd.DataFrame([row.replace(' ', '').split('|') for row in users.split('\n')])

In [45]:
users

,0,1,2
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [50]:
users_columns = 'users_id | banned | role'.split(' | ')
users_columns

['users_id', 'banned', 'role']

In [51]:
users.columns = users_columns

In [52]:
users

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [53]:
# Finally, our tables:

In [54]:
trips

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [55]:
users

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


### Query

* Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03". Round Cancellation Rate to two decimal points.
* Return the result table in any order

The **cancellation rate** is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

In [110]:
# Number of canceled requests

ps.sqldf(
    """

    SELECT request_at, 
           CAST(COUNT(*) AS float) AS num_canceled
    FROM trips LEFT JOIN users u1 ON trips.client_id = u1.users_id
               LEFT JOIN users u2 ON u2.users_id = trips.driver_id
    WHERE u1.banned = 'No' AND u2.banned = 'No' AND
          status LIKE "cancelled%" AND
          request_at BETWEEN "2013-10-01" AND "2013-10-03"
    GROUP BY request_at
    
    """
    )

,request_at,canceled
0,2013-10-01,1.0
1,2013-10-03,1.0


In [111]:
# Total number of requests

ps.sqldf(
    """

    SELECT request_at, 
           CAST(COUNT(*) AS float) AS num_total
    FROM trips LEFT JOIN users u1 ON trips.client_id = u1.users_id
               LEFT JOIN users u2 ON u2.users_id = trips.driver_id
    WHERE u1.banned = 'No' AND u2.banned = 'No' AND
          request_at BETWEEN "2013-10-01" AND "2013-10-03"
    GROUP BY request_at
    
    """
    )

,request_at,num_total
0,2013-10-01,3.0
1,2013-10-02,2.0
2,2013-10-03,2.0


In [116]:
# Calculation of cancellation rate

ps.sqldf(
    """

    WITH 
    total AS 
    (
        SELECT request_at, 
               CAST(COUNT(*) AS float) AS num_total
        FROM trips LEFT JOIN users u1 ON trips.client_id = u1.users_id
                   LEFT JOIN users u2 ON u2.users_id = trips.driver_id
        WHERE u1.banned = 'No' AND u2.banned = 'No' AND
              request_at BETWEEN "2013-10-01" AND "2013-10-03"
        GROUP BY request_at
    ),
    canceled AS
    (
        SELECT request_at, 
               CAST(COUNT(*) AS float) AS num_canceled
        FROM trips LEFT JOIN users u1 ON trips.client_id = u1.users_id
                   LEFT JOIN users u2 ON u2.users_id = trips.driver_id
        WHERE u1.banned = 'No' AND u2.banned = 'No' AND
              status LIKE "cancelled%" AND
              request_at BETWEEN "2013-10-01" AND "2013-10-03"
        GROUP BY request_at
    )
SELECT 
    total.request_at AS Day,
    CASE 
    WHEN (num_canceled / num_total) IS NULL THEN 0
    ELSE ROUND((num_canceled / num_total), 2) 
    END AS "Cancellation Rate"
FROM 
    total LEFT JOIN canceled ON total.request_at = canceled.request_at
    
    """
    )




,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50
